In [1]:
import gspread

gc = gspread.service_account(filename='./oauth.json')
print("Login successful")

Login successful


In [2]:
import pandas as pd
from src.tournament import Table, Player

worksheet = gc.open('Sample Data Spreadsheet').worksheet("Games")

games = pd.DataFrame.from_records(worksheet.get("Games"))
players = pd.DataFrame.from_records(worksheet.get("Players"))
waiting_players_list = pd.DataFrame.from_records(worksheet.get("WaitingPlayers"))

games.head(10)

,0,1,2,3
0,Dale,Dan,Erwin,Rudy
1,Glynda,Clarinda,Clarinda,Veronika
2,Moses,Veronika,Zofia,Terese
3,Olevia,Glennis,Jimmie,Glennis
4,Jolynn,Romana,Kaycee,Veronika
5,Libbie,Karissa,Bud,Alethea
6,Olevia,Emile,Louann,Emile
7,Catharine,Mellissa,Mozelle,Harriette
8,Rudy,Lachelle,Alethea,Joseph
9,Terese,Kaycee,Bud,Glynda


In [3]:
from src.tournament import Table, Player

player_history = {}
def init_player_history(name):
    player_history[name] = Player(name)
    
waiting_players = {}
def init_waiting_players(name):
    waiting_players[name] = True

for i, player in players.iterrows():
    init_player_history(player[0])

games_with_player = games.applymap(lambda x: player_history[x])
games_with_tables = games_with_player.apply(lambda game: Table(players=game), axis=1)

games_with_tables.head(10)

print(f'Tables = {games_with_tables.shape[0]}, Players = {len(players)}')

Tables = 557, Players = 50


In [4]:
def update_player_histories(table):
    for player in table:
        player.play_table(table)
    return table

tabled_games = games_with_tables.apply(update_player_histories)

waiting_player_history = []

for i, player in waiting_players_list.iterrows():
    waiting_player_history.append(player_history[player[0]])

waiting_players_list.head(10)

,0
0,Lucila
1,Jolynn
2,Joseph
3,Jimmie
4,Dale
5,Alethea
6,Onita
7,Louann
8,Lynsey
9,Mamie


In [7]:
from src.meta_heuristics import simulated_annealing_ensemble

result, cost = simulated_annealing_ensemble(waiting_player_history, 4, 500, 500)

print(f'Best cost = {cost}')
print(result)



Best cost = 26.57765601225993
[[Latoyia, Allena, Madeleine, Jimmie], [Ella, Clarinda, Terese, Romana], [Lucila, Alethea, Dale, Lynsey], [Nelida, Kaycee, Ellamae, Mamie], [Joseph, Rudy, Onita, Louann], [Jolynn, Erwin, See]]
